In [1]:
import subprocess
import os
import sys
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

def shelltest(CMD):
    """Executes a shell command and returns the result."""
    try:
        result = subprocess.check_output(CMD, shell=True, stderr=subprocess.STDOUT)
        return 0, result.decode('utf-8')
    except subprocess.CalledProcessError as e:
        logger.error("Shell command failed with CalledProcessError: %s", str(e))
        return e.returncode, e.output.decode('utf-8')
    except Exception as e:
        logger.exception("Unexpected error when running shell command.")
        return -1, str(e)

ERRORS = 0

## Test if pip package can be installed

In [2]:
TEST = "PIP PACKAGE INSTALLATION"
CMD = "pip install --user pytest || echo 'PIP Install: No'"
ret_code, output = shelltest(CMD)
logger.info(f"Output of {TEST}:\n{output}")

if 'PIP Install: No' in output or ret_code != 0:
    ERRORS += 1
    logger.error("PIP package installation failed.")

2023-10-18 00:54:48,912 - INFO - Output of PIP PACKAGE INSTALLATION:



## Test if conda directories are read/writable

In [3]:
def check_permissions(dir_path):
    """Check if a directory is readable and writable."""
    try:
        readable = os.access(dir_path, os.R_OK)
        writable = os.access(dir_path, os.W_OK)
        return readable, writable
    except Exception as e:
        logger.exception(f"Error checking permissions for {dir_path}.")
        return False, False

# Identify the conda directories
conda_base_dir = os.path.abspath(os.path.join(os.path.dirname(os.sys.executable), ".."))
conda_env_dir = os.environ.get('CONDA_PREFIX', '')

unaccessible_dirs = 0

# Ensure conda directories are available
if not (conda_base_dir or conda_env_dir):
    logger.error("Conda directory not found. Skipping test.")
    ERRORS += 1

for dir_name, dir_path in [('Conda Base Directory', conda_base_dir), 
                           ('Conda Environment Directory', conda_env_dir)]:
    readable, writable = check_permissions(dir_path)
    logger.info(f"{dir_name}: {dir_path}")
    logger.info(f"Readable: {readable}")
    logger.info(f"Writable: {writable}\n")

    if not (readable and writable):
        unaccessible_dirs += 1

if unaccessible_dirs == 2:
    ERRORS += 1
    logger.error("Both Conda directories are not fully accessible.")

2023-10-18 00:54:48,922 - INFO - Conda Base Directory: /opt/conda
2023-10-18 00:54:48,923 - INFO - Readable: True
2023-10-18 00:54:48,923 - INFO - Writable: True

2023-10-18 00:54:48,924 - INFO - Conda Environment Directory: 
2023-10-18 00:54:48,924 - INFO - Readable: False
2023-10-18 00:54:48,924 - INFO - Writable: False



## Report

In [4]:
if ERRORS > 0:
    logging.error(f"THERE ARE {ERRORS} ERRORS")
else:
    logging.info("ALL TESTS PASS")

logging.info(f"ERRORS: {ERRORS}")

2023-10-18 00:54:48,929 - INFO - ALL TESTS PASS
2023-10-18 00:54:48,930 - INFO - ERRORS: 0
